In [ ]:
import sys
print(sys.executable)
print(sys.version)


In [ ]:
!nvidia-smi

In [ ]:
from transformers import BitsAndBytesConfig
import os
from datasets import load_dataset

# Data

In [ ]:
!pwd

In [ ]:
trn_data = %pwd
trn_data = trn_data.replace('Notebooks','Files/training_set.jsonl')
val_data = trn_data.replace('training_set','validation_set')
print(trn_data)      
print(val_data)      
print(os.path.exists(trn_data))
print(os.path.exists(val_data))


In [ ]:
## Convert to huggingface dataset

data = load_dataset("json", data_files={
    "train":trn_data,
    "valid":val_data
})
data
                    

In [ ]:
data['train'][:5]

In [ ]:
data['valid'][:5]

In [ ]:
data['train'][0]['messages']

# Model

In [ ]:
# https://huggingface.co/ibm-granite/granite-4.0-h-1b

In [ ]:
from huggingface_hub import login
from dotenv import load_dotenv

In [ ]:
load_dotenv('/app/cloned_repo/LLM-World/.env')

In [ ]:
login(token=os.getenv('HF_TOKEN'))

In [ ]:
file_dir = %pwd
outp_dir = file_dir.replace('Notebooks','Files/sm_output')
outp_dir

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import get_peft_model, LoraConfig, TaskType

In [ ]:
model_id = "ibm-granite/granite-4.0-h-1b"

tokenizer = AutoTokenizer.from_pretrained(model_id) # Load Tokenizer
model = AutoModelForCausalLM.from_pretrained(model_id) # Load base model

In [ ]:
## Serialization (dict -> str) for ability for model to read (flatten)
def serialize_message(input_example):
    chat_str = ""
    for message in input_example['messages']:
        role = message['role']
        content = message['content']
        if role == "system":
            chat_str += f"System: {content}\n"
        elif role == "user":
            chat_str += f"User: {content}\n"
        elif role == "assistant":
            chat_str += f"Assistant: {content}\n"
    input_example["text"] = chat_str
    return input_example
        

In [ ]:
## Tokenize the data
def tokenize_text(example):
    return tokenizer(
        example['text'],
        padding='max_length',
        truncation=True,
        max_length=512)

In [ ]:
data = data.map(serialize_message)
data

In [ ]:
data['train'][0]

In [ ]:
data = data.map(tokenize_text,batched=True)
data

In [ ]:
data['train'][0]

In [ ]:
# HugingFace Trainer (Basic setup)
from transformers import Trainer, TrainingArguments

train_args = TrainingArguments(
    output_dir=outp_dir,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=8,
    eval_strategy="steps",
    save_steps=500,
    logging_steps=100,
    num_train_epochs=3,
    learning_rate=2e-5,
    # bf16=True,
    no_cuda=True, # dont use gpu when setting up variables If not using LoRA
    # use_cpu=True, # dont use gpu when setting up variables If not using LoRA
    fp16=True,
    push_to_hub=False,
    hub_model_id=None,
    hub_token=None
)

# better for training 1B+ OOM error possible without
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    task_type=TaskType.CAUSAL_LM,
    target_modules=["q_proj","v_proj"]
)

model = get_peft_model(model,lora_config)

trainer = Trainer(
    model=model,
    args=train_args,
    train_dataset=data['train'],
    eval_dataset=data['valid'],
    processing_class=tokenizer
)

In [ ]:
save_dir = file_dir.replace('Notebooks','Files/sm_artifacts')
save_dir

In [ ]:
trainer.train()
trainer.save_model(save_dir)
trainer.tokenizer.save_pretrained(save_dir)

## With TRL

In [1]:
from datasets import load_dataset
from loguru import logger

/app/home/marfok/LLM-World/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
trn_data = %pwd
trn_data = trn_data.replace('Notebooks','Files/training_set.jsonl')
val_data = trn_data.replace('training_set','validation_set')

In [3]:
## Convert to huggingface dataset

data = load_dataset("json", data_files={
    "train":trn_data,
    "valid":val_data
})
data
                    

DatasetDict({
    train: Dataset({
        features: ['messages'],
        num_rows: 409
    })
    valid: Dataset({
        features: ['messages'],
        num_rows: 20
    })
})

In [4]:
print(len(data['train']['messages']))
data['train']['messages'][0] # Conversation 1

409


[{'role': 'system',
  'content': 'You are a mathematician who is specialized in linear algebra and also statistics.'},
 {'role': 'user', 'content': 'What is a vector space in linear algebra?'},
 {'role': 'assistant',
  'content': 'A vector space is a collection of vectors where you can add them together and multiply them by scalars, following specific rules.'}]

In [5]:
## Serialization (dict -> str) for ability for model to read (flatten)
def serialize_message(input_example):
    chat_str = ""
    for message in input_example['messages']:
        role = message['role']
        content = message['content']
        if role == "system":
            chat_str += f"System: {content}\n"
        elif role == "user":
            chat_str += f"User: {content}\n"
        elif role == "assistant":
            chat_str += f"Assistant: {content}\n"
    input_example["text"] = chat_str
    return input_example
        

In [6]:
# Group by conversation so chat template can be used (alpac)
flat_train_data = data['train'].map(serialize_message)
flat_train_data 

Dataset({
    features: ['messages', 'text'],
    num_rows: 409
})

In [7]:
print(len(flat_train_data['text']))
flat_train_data['text'][0]

409


'System: You are a mathematician who is specialized in linear algebra and also statistics.\nUser: What is a vector space in linear algebra?\nAssistant: A vector space is a collection of vectors where you can add them together and multiply them by scalars, following specific rules.\n'

In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

In [9]:
model_name = "ibm-granite/granite-4.0-h-1b"


logger.info("SFT with TRL")
logger.info(f"Loading model: {model_name}")

2026-01-17 11:46:33.009 | INFO     | __main__:<module>:4 - SFT with TRL
2026-01-17 11:46:33.010 | INFO     | __main__:<module>:5 - Loading model: ibm-granite/granite-4.0-h-1b


In [10]:
model_id = model_name

tokenizer = AutoTokenizer.from_pretrained(model_id) # Load Tokenizer
logger.info(f"pad_token:{tokenizer.pad_token} (allows tokens to have the same length)")

if tokenizer.pad_token is None:    
    tokenizer.pad_token = tokenizer.eos_token
    logger.info(f"set pad_token=eos_token:{tokenizer.eos_token} (allows tokens to have the same length)")

# model = AutoModelForCausalLM.from_pretrained(model_id,device_map="auto",dtype=torch.bfloat16) # Load base model
model = AutoModelForCausalLM.from_pretrained(model_id) # Load base model

2026-01-17 11:46:34.445 | INFO     | __main__:<module>:4 - pad_token:<|pad|> (allows tokens to have the same length)
The fast path is not available because one of `(selective_state_update, causal_conv1d_fn, causal_conv1d_update)` is None. Falling back to the naive implementation. To install follow https://github.com/state-spaces/mamba/#installation and https://github.com/Dao-AILab/causal-conv1d


In [18]:
model

GraniteMoeHybridForCausalLM(
  (model): GraniteMoeHybridModel(
    (embed_tokens): Embedding(100352, 1536, padding_idx=100256)
    (layers): ModuleList(
      (0-4): 5 x GraniteMoeHybridDecoderLayer(
        (input_layernorm): GraniteMoeHybridRMSNorm((1536,), eps=1e-05)
        (post_attention_layernorm): GraniteMoeHybridRMSNorm((1536,), eps=1e-05)
        (shared_mlp): GraniteMoeHybridMLP(
          (activation): SiLUActivation()
          (input_linear): Linear(in_features=1536, out_features=8192, bias=False)
          (output_linear): Linear(in_features=4096, out_features=1536, bias=False)
        )
        (mamba): GraniteMoeHybridMambaLayer(
          (act): SiLUActivation()
          (conv1d): Conv1d(3328, 3328, kernel_size=(4,), stride=(1,), padding=(3,), groups=3328)
          (in_proj): Linear(in_features=1536, out_features=6448, bias=False)
          (norm): GraniteMoeHybridRMSNormGated()
          (out_proj): Linear(in_features=3072, out_features=1536, bias=False)
        )


In [19]:
model.config.use_cache = False # required for gradient checkpointing

In [20]:
from peft import LoraConfig, get_peft_model

In [21]:
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=[
        "q_proj", "k_proj","v_proj","o_proj"],
    bias="none",
    task_type="CAUSAL_LM"
)
pt_model = get_peft_model(model,peft_config)
pt_model.gradient_checkpointing_enable()
pt_model

/app/home/marfok/LLM-World/.venv/lib/python3.11/site-packages/peft/mapping_func.py:72: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/app/home/marfok/LLM-World/.venv/lib/python3.11/site-packages/peft/tuners/tuners_utils.py:285: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GraniteMoeHybridForCausalLM(
      (model): GraniteMoeHybridModel(
        (embed_tokens): Embedding(100352, 1536, padding_idx=100256)
        (layers): ModuleList(
          (0-4): 5 x GraniteMoeHybridDecoderLayer(
            (input_layernorm): GraniteMoeHybridRMSNorm((1536,), eps=1e-05)
            (post_attention_layernorm): GraniteMoeHybridRMSNorm((1536,), eps=1e-05)
            (shared_mlp): GraniteMoeHybridMLP(
              (activation): SiLUActivation()
              (input_linear): Linear(in_features=1536, out_features=8192, bias=False)
              (output_linear): Linear(in_features=4096, out_features=1536, bias=False)
            )
            (mamba): GraniteMoeHybridMambaLayer(
              (act): SiLUActivation()
              (conv1d): Conv1d(3328, 3328, kernel_size=(4,), stride=(1,), padding=(3,), groups=3328)
              (in_proj): Linear(in_features=1536, out_features=6448, bias=False)
              (

In [22]:
# trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
# total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in pt_model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in pt_model.parameters())


print(trainable_params)
print(total_params)

logger.info(f"Trainable parameters: {trainable_params:,}/{total_params:,} ({100*trainable_params/total_params:.2f}%)")

2026-01-17 11:47:49.256 | INFO     | __main__:<module>:10 - Trainable parameters: 655,360/1,462,193,728 (0.04%)


655360
1462193728


In [23]:
from trl import SFTTrainer, SFTConfig

In [24]:
pwd_dir = %pwd
file_dir = pwd_dir.replace('Notebooks','Files/granite_ckp')
logger.info(f"output_dir={file_dir}")

2026-01-17 11:47:53.553 | INFO     | __main__:<module>:3 - output_dir=/app/home/marfok/LLM-World/Files/granite_ckp


In [14]:
sft_training_args = SFTConfig(
    output_dir=str(file_dir),
    num_train_epochs=4,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    lr_scheduler_type="cosine",
    learning_rate=2e-5,
    max_length=2048,
    warmup_ratio=0.1,
    logging_steps=10,
    save_strategy="epoch",
    save_total_limit=2,
    bf16=True,
    optim="adamw_torch",
    seed=42,
    gradient_checkpointing=True,
    report_to="none",#"wandb",
    packing=False,
    assistant_only_loss=False
)


trainer = SFTTrainer(
    model = pt_model,
    processing_class=tokenizer,
    train_dataset=flat_train_data,
    # peft_config=peft_config,
    args=sft_training_args
)

In [15]:
trainer.train()

Step,Training Loss
10,4.903500
20,4.789900
30,4.442800
40,4.182600
50,3.926500
60,3.655600
70,3.496400
80,3.352000
90,3.346200
100,3.310500


TrainOutput(global_step=104, training_loss=3.911779146928054, metrics={'train_runtime': 1051.0639, 'train_samples_per_second': 1.557, 'train_steps_per_second': 0.099, 'total_flos': 1362585787904640.0, 'train_loss': 3.911779146928054})

In [ ]:
pt_model.save_pretrained(file_dir)
tokenizer.save_pretrained(file_dir)

In [ ]:
checkpoint_info = {
    "model_name":model_name,
    "output_dir":file_dir,
    "num_epochs":epochs,
    "total_examples":len(dataset),
    "loar_config":{"r":LORA_R,"alpha":LORA_ALPHA, "target_modules":TARGET_MODULES},
    "training_config":{"batch_size":batch_size, "gradient_accumulation":gradient_accululation,"learning_rate":learning_rate,"max_length":max_length}
    }

info_path = file_dir/"checkpont_info.json"

with open(info_path,"w") as f:
    json.dump(checkpoint_info, f, indent=2)

logger.info(f"Checkpoint info saved to {info_path}")

logger.info(f"Model saved to {file_dir}")

In [ ]:
import os
os.path.join(file_dir,'hi')

In [ ]:
## Load model
from peft import PeftModel
chk_dir = pwd.replace
ft_model = 